In [ ]:
!pip install imageio

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import numpy as np
import scipy
import scipy.misc
import tensorflow as tf
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import os
import zipfile
import requests
import imageio
from io import BytesIO

tf.logging.set_verbosity(tf.logging.INFO)
%matplotlib inline

In [ ]:
class Face():
  
  faceshapes = [
    'Oblong', 'Square', 'Round',
    'Square', 'Oblong', 'Heart', 'Square', 'Square', 'Square', 
    'Heart', 'Square', 'Round', 'Oblong', 'Heart','Square', 'Heart',
    'Square', 'Heart', 'Round','Oblong', 'Heart','Round', 'Oblong', 
    'Oblong', 'Heart', 'Square', 'Square', 'Oblong',
    'Square', 'Round', 'Round', 'Heart', 'Oblong', 'Oblong',
    'Square', 'Square', 'Square', 'Square', 'Oblong', 'Square',
  ] #why are there so many repeats of each faceshape? Oh wait this is the aggregated answers of the quiz right

  faceshape2int = {'Oblong' : 0, 'Square' : 1, 'Round' : 2, 'Heart' : 3} #rip Oval
  int2faceshape = {0 : 'Oblong', 1 : 'Square', 2 : 'Round', 3 : 'Heart'}

  allfaces = dict()
  
  
  def __init__(self, filename, image, shape):
    self.id = int(filename[:2])
    self.pose = int(filename[3])
    self.gender = filename[4]
    self.shape = Face.faceshapes[self.id-1]
    self.shapeid = Face.faceshape2int[self.shape]
    self.image = image
    self.image = scipy.misc.imresize(self.image, 1/8)
    self.image = self.image.reshape(self.image.shape[0] * self.image.shape[1])
    #self.image = scipy.misc.imresize(mpimg.imread('imm/' + filename + '.jpg'), 1/8)
    #self.image = self.image.reshape(self.image.shape[0] * self.image.shape[1])
    self.image = self.image.astype(np.float16)
    
    Face.allfaces[filename] = self
    
  @classmethod
  def get_equal_genders(self, pct):
        m = [x for x in self.allfaces.values() if x.gender == 'm']
        f = [x for x in self.allfaces.values() if x.gender == 'f']
        minlen = len(f)
        if len(m) < minlen:
            minlen = len(m)
        
        merged = np.array(m[:minlen] + f[:minlen])
        np.random.shuffle(merged)
        pct = int(2*minlen*pct)
        
        data = np.array([x.image for x in merged]).astype(np.float32)
        labels = np.array([1 if x.gender == 'm' else 0 for x in merged])
        
        return (data[:pct], data[pct:]), (labels[:pct], labels[pct:])
        
  @classmethod
  def get_train_test_split(self, pct):
    f = np.array([x for x in self.allfaces.values()])
    np.random.shuffle(f)
    pct = int(f.shape[0] * pct)
    
    data = np.array([x.image for x in f]).astype(np.float32)
    labels = np.array([x.shapeid for x in f]).astype(int)
    
    return (data[:pct], data[pct:]), (labels[:pct], labels[pct:])
    
        
        
        
    
  
  

In [ ]:
r = requests.get('http://www.imm.dtu.dk/~aam/datasets/imm_face_db.zip')
z = zipfile.ZipFile(BytesIO(r.content))
files = {file[:-4] for file in z.namelist() if file.endswith('asf')}

In [ ]:
faces = dict()
for file in files:
    f = Face(file, imageio.imread(z.extract(file + '.jpg'), pilmode ='F', as_gray=True), Face.faceshapes[int(file[:2])-1])
    faces[file] = f

In [ ]:
(train_data, test_data), (train_labels, test_labels) = Face.get_train_test_split(0.75)

In [ ]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 60, 80, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 15 * 20 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=4)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
eval_data = test_data
eval_labels = test_labels

In [ ]:
gender_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn,
    model_dir='/tmp/facemodel/'
)

tensors_to_log = {'probabilities': 'softmax_tensor'}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50
)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x' : train_data},
    y=train_labels,
    batch_size=10,
    num_epochs=100,
    shuffle=True
)


In [ ]:
# Don't train again, we have a good accuracy >_> <_<
#
gender_classifier.train(
    input_fn=train_input_fn,
    steps=1000,
    hooks=[logging_hook]
)

In [ ]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False
)

eval_results = gender_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

In [ ]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x' : eval_data},
    num_epochs=1,
    shuffle=False
)

pred_results = gender_classifier.predict(input_fn=predict_input_fn)

In [ ]:
def facedatabase(shape_class):
  print('You have a {} shape'.format(Face.int2faceshape[shape_class]))
  if shape_class == 0:
      print('\nFor Women: \n\t Bob with sideswept bangs, Loose Waves, Layered Hair, Lob')
      print('For Men: \n\t Let it grow out to about an inch on the top and sides and ask the barber to add some texture to it\n\n')
  elif shape_class == 1: 
    print('\nFor Women: \n\t Lob, A Center Part with Long Layers, Pixie, Long Layers')
    print('For Men: \n\t Go for a short back and side with some length on top, ideally a couple of inches as your hair is likely to stick up. Keep it shorter so you can control it\n\n')
  elif shape_class == 2:
    print('\nFor Women: \n\t Sleek Middle Part, Long Waves, Undercut, Middle Length Lob')
    print('For Men: \n\t Short back and sides with around 3 to 5 inches of length through to front to create the height needed. Ask your barber for a quiff and get some height with your hairdryer\n\n')
  elif shape_class == 3:
    print('\nFor Women: \n\t Long Picey Bangs, Deep Side Part, Blunt Wispy Bangs, Sleek Crop')
    print('For Men: \n\t Mid-Length Swept Look, A Beard\n\n')

In [ ]:
for p in pred_results:
    fa = p['classes']
    print(p)

In [ ]:
facedatabase(0)